In [3]:
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
from functools import partial

# Step 1: Set the video filenames
video_filenames = ["fancy.mp4", "feel.mp4", "fancy.mp4", "feel.mp4"]

# Step 2: Create VideoFileClip objects from the videos
video_clips1 = []
for filename in video_filenames:
    video = VideoFileClip(filename).subclip(0,3)
    video = video.resize(height=1080)
    video = video.set_position(("center", "center"))  # Set video position to center
    video_clips1.append(video)


color_list = ['BONE', 'OCEAN', 'quiet', 'DEEPGREEN', 'HOT', 'COOL', 'PINK',
              'INFERNO', 'CIVIDIS', 'RAINBOW', 'TWILIGHT', 'TURBO', 'BGR2RGB']


def color_clips_lists(video_clips,color_lists):
    # Create a list to store the color-graded video clips
    color_clips_list = []

    # Define the apply_colormap function
    def apply_colormap(frame, color):
        # Convert the frame to grayscale and then to 8-bit depth
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray_frame_8bit = cv2.convertScaleAbs(gray_frame)

        # Apply the specified colormap from color_list
        colormap = getattr(cv2, f'COLORMAP_{color}')
        colored_frame = cv2.applyColorMap(gray_frame_8bit, colormap)

        return colored_frame

    # Iterate through video clips and color list
    for video_clip, color in zip(video_clips, color_lists):
        if color.lower() == 'quiet':
            def bgr_to_rgb(frame):
                return frame[..., ::-1]  # Reverse BGR to RGB

            # Apply bgr_to_rgb function if color is 'none'
            color_clip = video_clip.fl_image(bgr_to_rgb)
        else:
            # Use functools.partial to create a partially applied function
            apply_colormap_partial = partial(apply_colormap, color=color)
            color_clip = video_clip.fl_image(apply_colormap_partial)

        # Add the color-graded clip to the list
        color_clips_list.append(color_clip)
    return color_clips_list


# Create color-graded video clips
color_clips = color_clips_lists(video_clips1, color_list)

# # Concatenate the color-graded video clips into one video
# final_video = concatenate_videoclips(color_clips, method="compose")

# # Export the final video to "color0.mp4"
# final_video.write_videofile("color0.mp4", codec="libx264", fps=24)


In [5]:

from moviepy.editor import *
video_clips = color_clips
padding = 0.6
video_fx_list = [video_clips[0]]

idx = video_clips[0].duration - padding
for video in video_clips[1:]:
    video_fx_list.append(video.set_start(idx).crossfadein(padding))
    idx += video.duration - padding

final_video = CompositeVideoClip(video_fx_list)
# Concatenate the color-graded video clips into one video
final_video = concatenate_videoclips([final_video,video_clips1[1]], method="compose")
# Export the final video to "color0.mp4"
# final_video.write_videofile("color0.mp4", codec="libx264", fps=24)

In [3]:
# Get the frames from the final_video
frames = final_video.get_frame(0)  # Get the first frame to determine frame size
frame_size = frames.shape[1], frames.shape[0]

# Define the output video file path
output_video_path = "color1.mp4"

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can change the codec as needed
out = cv2.VideoWriter(output_video_path, fourcc, 24.0, frame_size)

# Iterate through the frames and write them to the output video
for frame in final_video.iter_frames(fps=24):
    out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

# Release the VideoWriter and close the output file
out.release()

print("Video written with cv2 successfully.")

Video written with cv2 successfully.


In [1]:
# Initialize an empty list to store frames and audio
frames = []
audio = []
from tqdm import tqdm
# Estimate the total number of frames
total_frames = int(final_video.duration * final_video.fps)

# Iterate through the frames and append each frame as a NumPy array
for frame in tqdm(final_video.iter_frames(fps=final_video.fps, dtype='uint8'), total=total_frames):
    frames.append(frame)
    audio.append(final_video.audio.get_frame(final_video.duration * len(frames) / total_frames))

# Convert the list of frames to a NumPy array
combined_array = np.array(frames)

# Get video dimensions
height, width, _ = combined_array[0].shape

# Create a VideoWriter object to save the video as an MP4 file
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
output_video = cv2.VideoWriter('dahyuncv2.mp4', fourcc, final_video.fps, (width, height))

# Write each frame to the output video
for frame in tqdm(combined_array, total=total_frames):
    output_video.write(frame)

# Release the output video
output_video.release()

# Combine the audio frames into a NumPy array
audio_array = np.array(audio)

# Save the audio as a WAV file with a constant sample rate (e.g., 44100 Hz)
audio_file = "combined_audio.wav"
final_video.audio.to_audiofile(audio_file, codec="pcm_s16le", fps=44100)

# Merge the video and audio
merged_file = "color2.mp4"
command = f"ffmpeg -i dahyuncv2.mp4 -i {audio_file} -c:v copy -c:a aac -strict experimental -y {merged_file}"
os.system(command)


In [ ]:
import numpy as np
from moviepy.editor import VideoClip
from moviepy.video.io.ffmpeg_tools import np2vid

# Assuming you already have combined_array as a NumPy array of frames

# Define a custom function to generate frames
def frame_generator(t):
    frame_index = int(t * final_video.fps)
    if frame_index < len(combined_array):
        return combined_array[frame_index]
    else:
        return combined_array[-1]

# Create a VideoClip using the custom frame generator
video_clip = VideoClip(frame_generator, duration=final_video.duration)

# Now you have a moviepy VideoClip named video_clip without audio
